# 2.0 Join data

AIMS

En este archivo se realiza la union de los datos para formar un unico dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from mypackage import dir
from mypackage.transforms import convert_variables
from mypackage.calculates import proportion_categories, df_correlations
from rfm import calculate_rfm

modality =  modality = 'p'
project = 'australian'
data = dir.make_dir_line(modality, project) 
processed = data('processed')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

## 2.1 Load data

In [3]:
df_customers = pd.read_parquet(processed/'df_clientes.parquet.gzip')
df_customers = df_customers.loc[:,['id_cliente', 'sexo', 'age_days']]
df_customers = convert_variables(df_customers, ['id_cliente', 'sexo'], 'str')
df_customers.head()

,id_cliente,sexo,age_days
0,1001,1,34263
1,1002,1,30482
2,1003,0,39372
3,1004,0,14288
4,1005,0,2810


In [4]:
df_products = pd.read_parquet(processed/'df_productos.parquet.gzip')
df_products = df_products.loc[:,['id_producto', 'category', 'valor', 'valor_produccion']]
df_products.head()

,id_producto,category,valor,valor_produccion
0,759ne,Short,40.0,22.4
1,759bl,Short,40.0,22.4
2,759az,Short,40.0,22.4
3,759ro,Short,40.0,22.4
4,759ra,Short,40.0,22.4


In [5]:
df_sales = pd.read_parquet(processed/'df_ventas.parquet.gzip')
df_sales.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2
0,1001,2021-02-17,1088,2521ro,1,0,0,0.0
1,1002,2021-02-21,1089,9160ro,2,2,1,5.0
2,1003,2021-02-22,1050,9173ne,2,0,0,0.0
3,1004,2021-02-24,1029,762bl,1,1,0,0.0
4,1005,2021-02-24,1009,9167ro,1,1,0,0.0


In [6]:
print(df_sales.shape, df_products.shape)
df = df_sales.merge(df_products, on = ['id_producto'])
print(df.shape)

(5447, 8) (361, 4)
(5447, 11)


In [7]:
df['total_factura'] = df['valor'] * df['quantity']
df['total_produccion'] = df['valor_produccion'] * df['quantity']
df['total_descuento'] = (df['total_factura'] * df['per_discount2'])/100
df['total_factura_con_descuento'] = df['total_factura'] - df['total_descuento'] 
df['ganancia'] = df['total_factura_con_descuento'] - df['total_produccion'] 
df.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2,category,valor,valor_produccion,total_factura,total_produccion,total_descuento,total_factura_con_descuento,ganancia
0,1001,2021-02-17,1088,2521ro,1,0,0,0.0,Capri,52.8,13.2,52.8,13.2,0.00,52.80,39.60
1,1226,2021-04-07,1008,2521ro,2,0,0,0.0,Capri,52.8,13.2,105.6,26.4,0.00,105.60,79.20
2,2703,2021-12-25,1022,2521ro,1,1,0,0.0,Capri,52.8,13.2,52.8,13.2,0.00,52.80,39.60
3,2837,2022-03-01,1099,2521ro,2,0,1,15.0,Capri,52.8,13.2,105.6,26.4,15.84,89.76,63.36
4,2991,2022-04-03,1048,2521ro,1,1,1,5.0,Capri,52.8,13.2,52.8,13.2,2.64,50.16,36.96


## 2.2 Make some calculations

In [19]:
datos_compras = {
    'fecha_compra': ['2024-01-01', '2024-01-03', '2024-01-06', '2024-01-10', '2024-01-15'],
    # Otras columnas de datos de compras...
}

df_compras = pd.DataFrame(datos_compras)
df_compras.sort_values(by='fecha_compra', inplace=True)
df_compras['fecha_compra'] = pd.to_datetime(df_compras['fecha_compra'])
df_compras['tiempo_entre_compras'] = df_compras['fecha_compra'].diff()
df_compras = df_compras.drop(index=0)
df_compras

,fecha_compra,tiempo_entre_compras
1,2024-01-03,2 days
2,2024-01-06,3 days
3,2024-01-10,4 days
4,2024-01-15,5 days


In [20]:
tiempo_minimo = df_compras['tiempo_entre_compras'].min()
tiempo_promedio = df_compras['tiempo_entre_compras'].mean()
tiempo_maximo = df_compras['tiempo_entre_compras'].max()

# Crear un nuevo DataFrame con los resultados
df_resultado = pd.DataFrame({
'tiempo_minimo': [tiempo_minimo],
'tiempo_promedio': [tiempo_promedio],
'tiempo_maximo': [tiempo_maximo]
})

df_resultado

,tiempo_minimo,tiempo_promedio,tiempo_maximo
0,2 days,3 days 12:00:00,5 days


In [28]:
def calcular_tiempo_entre_compras(df):
  """
  Calcula el tiempo mínimo, promedio y máximo entre compras en un DataFrame.

  Parámetros:
    df: DataFrame con las compras. Debe tener una columna con la fecha de compra.

  Retorno:
    Nuevo DataFrame con las siguientes columnas:
      - 'tiempo_minimo': Tiempo mínimo entre compras.
      - 'tiempo_promedio': Tiempo promedio entre compras.
      - 'tiempo_maximo': Tiempo máximo entre compras.
  """

  # Convertir la columna de fecha a tipo datetime
  df['fecha_compra'] = pd.to_datetime(df['fecha_compra'])

  # Calcular la diferencia entre fechas consecutivas
  df['tiempo_entre_compras'] = df['fecha_compra'].diff()

  # Eliminar el primer valor (no hay diferencia para la primera compra)
  df = df.drop(index=0)

  # Calcular el tiempo mínimo, promedio y máximo
  tiempo_minimo = df['tiempo_entre_compras'].min()
  tiempo_promedio = df['tiempo_entre_compras'].mean()
  tiempo_maximo = df['tiempo_entre_compras'].max()

  # Crear un nuevo DataFrame con los resultados
  df_resultado = pd.DataFrame({
    'tiempo_minimo': [tiempo_minimo],
    'tiempo_promedio': [tiempo_promedio],
    'tiempo_maximo': [tiempo_maximo]
  })

  return df_resultado

# Ejemplo de uso
df_compras = pd.DataFrame({
  'fecha_compra': ['2023-01-01', '2023-01-15', '2023-02-01', '2023-03-01'],
  'valor': [100, 200, 300, 400]
})

df_resultado = calcular_tiempo_entre_compras(df_compras)
df_resultado


,tiempo_minimo,tiempo_promedio,tiempo_maximo
0,14 days,19 days 16:00:00,28 days


In [55]:
# Función para calcular el tiempo entre compras
def calcular_tiempo_entre_compras(df):
    """
    Calculates the minimum, average and maximum days between purchases in a DataFrame.

    Parámetros:
    df: DataFrame con las compras. Debe tener una columna con la fecha de compra.

    Retorno:
    Nuevo DataFrame con las siguientes columnas:
        - 'tiempo_minimo': Tiempo mínimo entre compras.
        - 'tiempo_promedio': Tiempo promedio entre compras.
        - 'tiempo_maximo': Tiempo máximo entre compras.
    """
    # Ordenar el DataFrame por cliente y fecha
    df = df.sort_values(by=['Cliente', 'Fecha'])

    # Calcular la diferencia de tiempo entre compras para cada cliente
    df['Tiempo_Entre_Compras'] = df.groupby('Cliente')['Fecha'].diff()
    df['Tiempo_Entre_Compras'] = df['Tiempo_Entre_Compras'].dt.days

    # Calcular los estadísticos
    stats = df.groupby('Cliente')['Tiempo_Entre_Compras'].agg(['min', 'mean', 'max'])

    # Crear un nuevo DataFrame con los resultados
    resultados_df = pd.DataFrame(stats)
    resultados_df.columns = ['Tiempo_Min', 'Tiempo_Promedio', 'Tiempo_Max']
    resultados_df['Tiempo_Promedio'] = round(resultados_df['Tiempo_Promedio'],0)
    resultados_df = resultados_df.reset_index()
    
    return resultados_df

# Ejemplo de DataFrame de compras
data = {
    'Cliente': ['A', 'A', 'A', 'A',
                'B', 'B', 
                'C', 'C', 'C', 'C'],
    'Fecha': ['2023-01-01', '2023-01-15', '2023-02-01', '2023-03-01',
              '2023-01-01', '2023-02-01', 
              '2022-01-01', '2023-01-06', '2023-02-01', '2023-06-01',]
}
compras_df = pd.DataFrame(data)
compras_df['Fecha'] = pd.to_datetime(compras_df['Fecha'])

# Calcular los tiempos entre compras
resultados_df = calcular_tiempo_entre_compras(compras_df)
resultados_df


,Cliente,Tiempo_Min,Tiempo_Promedio,Tiempo_Max
0,A,14.0,20.0,28.0
1,B,31.0,31.0,31.0
2,C,26.0,172.0,370.0


In [8]:
df_rfm = calculate_rfm(df, val_id_customer='id_cliente', 
                       val_id_facture = 'id_factura', 
                       val_money = 'ganancia', 
                       val_date = 'fecha')
df_rfm.head()

,id_cliente,recency,frequency,monetary
0,1001,5,53,2592.016
1,1002,5,48,3161.840
2,1003,63,55,3025.264
3,1004,39,63,3657.104
4,1005,58,53,3396.528


In [9]:
df_prop = proportion_categories(df, 
                                pet=0, 
                                val1='id_cliente', 
                                val2='category', 
                                val3='ganancia')
df_prop.head()

category,id_cliente,Babucha,Bermuda,Calza,Camisa,Capri,Chaleco,Jean,Mini,Musculosa,Pantalón,Pollera,Remera,Short,Strapless,Vestido
0,1001,0.000000,0.035714,0.053571,0.107143,0.053571,0.107143,0.107143,0.035714,0.214286,0.000000,0.000000,0.053571,0.142857,0.035714,0.053571
1,1002,0.020408,0.020408,0.000000,0.102041,0.102041,0.102041,0.163265,0.000000,0.224490,0.020408,0.000000,0.102041,0.142857,0.000000,0.000000
2,1003,0.035714,0.017857,0.000000,0.125000,0.071429,0.053571,0.142857,0.035714,0.196429,0.017857,0.000000,0.125000,0.107143,0.071429,0.000000
3,1004,0.015385,0.015385,0.000000,0.153846,0.107692,0.030769,0.092308,0.015385,0.261538,0.000000,0.015385,0.153846,0.123077,0.015385,0.000000
4,1005,0.000000,0.000000,0.017544,0.105263,0.105263,0.070175,0.192982,0.052632,0.210526,0.017544,0.000000,0.105263,0.035088,0.052632,0.035088


In [10]:
df_disc = proportion_categories(df, 
                                pet=0, 
                                val1='id_cliente', 
                                val2='discount', 
                                val3='ganancia')
df_disc.rename(columns={'0':'local', '1':'no_local'}, inplace=True)
df_disc.head()

discount,id_cliente,local,no_local
0,1001,0.500000,0.500000
1,1002,0.510204,0.489796
2,1003,0.517857,0.482143
3,1004,0.630769,0.369231
4,1005,0.421053,0.578947


## 2.3 Join

In [11]:
dataset = df_customers.copy()

In [12]:
print(dataset.shape, df_rfm.shape, df_prop.shape, df_disc.shape)
dataset = dataset.merge(df_rfm, on = ['id_cliente'])
print(dataset.shape)
dataset = dataset.merge(df_prop, on = ['id_cliente'])
print(dataset.shape)
dataset = dataset.merge(df_disc, on = ['id_cliente'])
print(dataset.shape)

(100, 3) (100, 4) (100, 16) (100, 3)
(100, 6)
(100, 21)
(100, 23)


In [13]:
dataset = dataset.loc[:,['id_cliente', 'sexo', 'age_days', 'recency', 'frequency', 'monetary',
       'Babucha', 'Bermuda', 'Calza', 'Camisa', 'Capri', 'Chaleco', 'Jean',
       'Mini', 'Pantalón', 'Pollera', 'Remera', 'Short',
       'Strapless', 'Vestido', 'local']]
dataset.head()

,id_cliente,sexo,age_days,recency,frequency,monetary,Babucha,Bermuda,Calza,Camisa,Capri,Chaleco,Jean,Mini,Pantalón,Pollera,Remera,Short,Strapless,Vestido,local
0,1001,1,34263,5,53,2592.016,0.000000,0.035714,0.053571,0.107143,0.053571,0.107143,0.107143,0.035714,0.000000,0.000000,0.053571,0.142857,0.035714,0.053571,0.500000
1,1002,1,30482,5,48,3161.840,0.020408,0.020408,0.000000,0.102041,0.102041,0.102041,0.163265,0.000000,0.020408,0.000000,0.102041,0.142857,0.000000,0.000000,0.510204
2,1003,0,39372,63,55,3025.264,0.035714,0.017857,0.000000,0.125000,0.071429,0.053571,0.142857,0.035714,0.017857,0.000000,0.125000,0.107143,0.071429,0.000000,0.517857
3,1004,0,14288,39,63,3657.104,0.015385,0.015385,0.000000,0.153846,0.107692,0.030769,0.092308,0.015385,0.000000,0.015385,0.153846,0.123077,0.015385,0.000000,0.630769
4,1005,0,2810,58,53,3396.528,0.000000,0.000000,0.017544,0.105263,0.105263,0.070175,0.192982,0.052632,0.017544,0.000000,0.105263,0.035088,0.052632,0.035088,0.421053


## 2.4 Correlations

In [14]:
salida_correlations = df_correlations(dataset.loc[:, dataset.columns != 'id_sample'], sort_by_distance=True)
salida_correlations

,Column_1,Column_2,Correlation,type
41,monetary,frequency,0.778599,Strong Positive
167,Short,Capri,-0.321089,Weak Negative
223,Remera,Mini,-0.262036,Weak Negative
105,Jean,Bermuda,-0.254167,Weak Negative
129,Short,Calza,-0.251491,Weak Negative
205,Short,Jean,-0.250942,Weak Negative
112,Vestido,Bermuda,0.232402,Weak Positive
204,Remera,Jean,-0.231700,Weak Negative
49,Mini,frequency,0.223149,Weak Positive
146,Pollera,Camisa,0.217810,Weak Positive


In [15]:
# Save data
dataset.to_parquet(processed/'dataset.parquet.gzip', compression='gzip')

In [16]:
print('ok_')

ok_
